<font size=6> <b> SMU Lab Three - MSDS7331 - Machine Learning-1 </b> </font>

<font size=5> <b> Summer 2020 Group - Sachin, Ikenna, Edgar, Dustin </b></font> 




<img src="https://github.com/sachinac/ML7331/blob/master/data/maxresdefault.png?raw=true"> 

<p align="center"><font size=5> <b> Movie Recommender System  </b></font> </p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1">Import Libraries</a></span></li><li><span><a href="#Business-Understanding-1" data-toc-modified-id="Business-Understanding-1-2">Business Understanding 1</a></span></li><li><span><a href="#Data-Understanding-1" data-toc-modified-id="Data-Understanding-1-3">Data Understanding 1</a></span></li><li><span><a href="#Data-Understanding-2" data-toc-modified-id="Data-Understanding-2-4">Data Understanding 2</a></span></li><li><span><a href="#Model-and-Evaluations-1" data-toc-modified-id="Model-and-Evaluations-1-5">Model and Evaluations 1</a></span></li><li><span><a href="#Model-and-Evaluations-2" data-toc-modified-id="Model-and-Evaluations-2-6">Model and Evaluations 2</a></span></li><li><span><a href="#Model-and-Evaluations-3" data-toc-modified-id="Model-and-Evaluations-3-7">Model and Evaluations 3</a></span></li><li><span><a href="#Model-and-Evaluations-4" data-toc-modified-id="Model-and-Evaluations-4-8">Model and Evaluations 4</a></span></li><li><span><a href="#Deployment" data-toc-modified-id="Deployment-9">Deployment</a></span></li><li><span><a href="#Exceptional-Work" data-toc-modified-id="Exceptional-Work-10">Exceptional Work</a></span></li><li><span><a href="#References" data-toc-modified-id="References-11">References</a></span></li></ul></div>

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import turicreate as tc
import ast
import wget
import zipfile
import os

timebreak = '\n\n----------------------------------------------------\nCell Processing Time'

# Business Understanding

Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?


# Data Understanding 1

Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?

## Download the data:
Because the unzipped data is ~ 900mb, we've saved it in an AWS S3 bucket as a zip file.  We've added the movies data directory to our .gitignore file so it doesn't get picked up in github.   This step will check if the movie directory has already been created, and skip the step if so.  If the directory doesn't exist, it creates the movie directory, checks files in the directory against expected files, and if not found, it downloads the zip file, unzips and deletes the zip file and cleans up created variables.

In [2]:
%%time
movie_dir = './data/movies'
movie_zipfile = movie_dir + '/movies.zip'
expected_files = ['keywords.csv', 'credits.csv', 'ratings.csv', 'ratings_small.csv', 'links_small.csv', 'movies_metadata.csv', 'links.csv']
movie_data = 'https://ml7331.s3.us-east-2.amazonaws.com/3405_6663_bundle_archive.zip'

# If movie directory doesn't exist, create & download movie data:
if not os.path.isdir(movie_dir):
    print('creating ', movie_dir)
    os.mkdir(movie_dir)
    
if not all(files in os.listdir(movie_dir) for files in expected_files):
    print('downloading files...')
    wget.download(movie_data, movie_zipfile)
    print('unzipping files...')
    with zipfile.ZipFile(movie_zipfile, 'r') as zip_ref:
        zip_ref.extractall(movie_dir)
    os.remove(movie_zipfile)
    
del movie_data, movie_zipfile, movie_dir, expected_files
print('Movie files are present!', timebreak)

creating  ./data/movies
downloading files...
unzipping files...
Movie files are present! 

----------------------------------------------------
Cell Processing Time
CPU times: user 5.12 s, sys: 1.26 s, total: 6.38 s
Wall time: 13.3 s


In [3]:
%%time
# Read in datasets
#TODO: these should go in their own sections eventually
links = pd.read_csv('./data/movies/links.csv')
links_small = pd.read_csv('./data/movies/links_small.csv')
ratings = pd.read_csv('./data/movies/ratings.csv')
ratings_small = pd.read_csv('./data/movies/ratings_small.csv')

CPU times: user 3.43 s, sys: 421 ms, total: 3.85 s
Wall time: 3.86 s


## Movie Credits

The credits dataset (found in /data/credits.csv) has a movie id column and provides two JSON objects stored as a string representing the crew and cast.  It uses single quotes for strings, except in cases where a single quote is used, these strings are wrapped in double quotes.  Well formed JSON expects strings to be wrapped in double quotes, so we'll write a little helper function using the ***literal_eval*** function from the Abstract Syntax Trees package.

After parsing them as literal into a list object, we can pick out directors, producers and actors, which we think will be most important in influencing recommendations.  We'll save these as list objects which we can then merge into our movies dataset.

In [4]:
%%time

credits = pd.read_csv('./data/movies/credits.csv')

# Drop rows where both cast and crew are empty
credits = credits[~((credits['cast'].str.contains('\[\]')) & (credits['crew'].str.contains('\[\]')))]

def get_credits(crew_json):
    crew = ast.literal_eval(crew_json.crew)
    cast = ast.literal_eval(crew_json.cast)
    directors = []
    producers = []
    actors = []
    for member in crew:
        if 'Director' in member['job']:
            directors.append(member['name'])
        if 'Producer' in member['job']:
            producers.append(member['name'])
    for actor in cast:
        actors.append(actor['name'])

    return  pd.Series([directors, producers, actors])

credits[['directors','producers','actors']] = credits.apply(get_credits, axis=1)


print(credits.info(),timebreak)

credits


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45055 entries, 0 to 45475
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cast       45055 non-null  object
 1   crew       45055 non-null  object
 2   id         45055 non-null  int64 
 3   directors  45055 non-null  object
 4   producers  45055 non-null  object
 5   actors     45055 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB
None 

----------------------------------------------------
Cell Processing Time
CPU times: user 36.9 s, sys: 200 ms, total: 37.1 s
Wall time: 37.1 s


,cast,crew,id,directors,producers,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[John Lasseter, Ash Brannon, William Reeves, R...","[Bonnie Arnold, Ed Catmull, Ralph Guggenheim, ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Joe Johnston, Thomas E. Ackerman]","[Larry J. Franco, Scott Kroopf, Ted Field, Rob...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch],[],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker],"[Ronald Bass, Ezra Swerdlow, Deborah Schindler...","[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[Elliot Davis, Charles Shyer]",[Nancy Meyers],"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,"[Hamid Nematollah, Babak Ardalan]",[Hamid Nematollah],"[Leila Hatami, Kourosh Tahami, Elham Korda]"
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,[Lav Diaz],[],"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,"[Mark L. Lester, João Fernandes]",[],"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,[Yakov Protazanov],[Joseph N. Ermolieff],"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


## Movie Keywords

The keywords dataset (found in /data/keywords.csv) has a movie id column and again provides a JSON object stored as a string representing keywords which apply to each movie.  It uses single quotes for strings, except in cases where a single quote is used, these strings are wrapped in double quotes.  Well formed JSON expects strings to be wrapped in double quotes, so we'll utilize another helper function using the ***literal_eval*** function from the Abstract Syntax Trees package.

After parsing them as literal into a list object, and removing empty list objects, we'll save these keywords as list object which we can merge into our movies dataset.

In [5]:
%%time

keywords = pd.read_csv('./data/movies/keywords.csv')

# Remove empty entries:
keywords = keywords[~keywords['keywords'].str.contains('\[\]')]

keywords['keyword_json'] = keywords['keywords'] 

# Breakout json into a list of names
def get_keywords(keyword_list):
    import ast
    kw = []
    klist = ast.literal_eval(keyword_list)
    for word in klist:
        kw.append(word['name'])
    return kw

keywords['keywords'] = keywords.keyword_json.apply(get_keywords)

print(keywords.info(),timebreak)

keywords

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31624 entries, 0 to 46415
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            31624 non-null  int64 
 1   keywords      31624 non-null  object
 2   keyword_json  31624 non-null  object
dtypes: int64(1), object(2)
memory usage: 988.2+ KB
None 

----------------------------------------------------
Cell Processing Time
CPU times: user 1.35 s, sys: 9.88 ms, total: 1.36 s
Wall time: 1.36 s


,id,keywords,keyword_json
0,862,"[jealousy, toy, boy, friendship, friends, riva...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[board game, disappearance, based on children'...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[fishing, best friend, duringcreditsstinger, o...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[based on novel, interracial relationship, sin...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[baby, midlife crisis, confidence, aging, daug...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...
46409,84419,"[revenge, murder, serial killer, new york city...","[{'id': 9748, 'name': 'revenge'}, {'id': 9826,..."
46410,390959,[blair witch],"[{'id': 224180, 'name': 'blair witch'}]"
46411,289923,"[witch, mythology, legend, serial killer, mock...","[{'id': 616, 'name': 'witch'}, {'id': 2035, 'n..."
46414,439050,[tragic love],"[{'id': 10703, 'name': 'tragic love'}]"


## Movie Metadata



The movies dataset (found in /data/movies_metadata.csv) has a movie id column and provides a lot of usable and JSON objects storing various metadata about each movie.  

In this section we'll accomplish the following:  
* There are three movies with improper ID values (1997-08-20,2012-09-29,2014-01-01), these items all appear to have incorrect formatting, so we'll drop these in order to merge the keyword and credits datasets and prevent future problems.
* Merge directors, producers and actors from credits dataset
* Merge keywords from keywords dataset
* Clean up the movie collection by converting it to a string, as it appears to only contain one item per movie.
* Convert the JSON genres object into a list of genres

In [6]:
%%time
movies = pd.read_csv('./data/movies/movies_metadata.csv', low_memory=False)
print('Original Movies Shape: ', movies.shape, '\n')

# Remove the movies with erroneous data & convert ID to int:
movies = movies[~movies['id'].str.contains('-')]
movies['id'] = movies['id'].astype(int)

print('Movies Shape after dropping bad data: ', movies.shape, '\n')

# Add Keywords & Credits
movies = movies.merge(keywords, how='left', on='id')
print('Movies Shape after merging keywords: ', movies.shape, '\n')

movies = movies.merge(credits, how='left', on='id')
print('Movies Shape after merging credits: ', movies.shape, '\n')

movies = movies.drop(columns=['cast','crew','keyword_json'])

# Function to convert collection to a string
def get_collection(json):
    if pd.isnull(json):
        return np.nan
    else:
        return ast.literal_eval(json)['name']

movies['belongs_to_collection'] = movies.belongs_to_collection.apply(get_collection)

#We can re-use our get_keywords function here to get genres
movies['genres'] = movies.genres.apply(get_keywords)

print(movies.info(),timebreak)

movies


Original Movies Shape:  (45466, 24) 

Movies Shape after dropping bad data:  (45463, 24) 

Movies Shape after merging keywords:  (46013, 26) 

Movies Shape after merging credits:  (46125, 31) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46125 entries, 0 to 46124
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  46125 non-null  object 
 1   belongs_to_collection  4546 non-null   object 
 2   budget                 46125 non-null  object 
 3   genres                 46125 non-null  object 
 4   homepage               7928 non-null   object 
 5   id                     46125 non-null  int64  
 6   imdb_id                46108 non-null  object 
 7   original_language      46114 non-null  object 
 8   original_title         46125 non-null  object 
 9   overview               45162 non-null  object 
 10  popularity             46121 non-null  object 
 11  poster_path     

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,keywords,directors,producers,actors
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,"[jealousy, toy, boy, friendship, friends, riva...","[John Lasseter, Ash Brannon, William Reeves, R...","[Bonnie Arnold, Ed Catmull, Ralph Guggenheim, ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[board game, disappearance, based on children'...","[Joe Johnston, Thomas E. Ackerman]","[Larry J. Franco, Scott Kroopf, Ted Field, Rob...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[fishing, best friend, duringcreditsstinger, o...",[Howard Deutch],[],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[based on novel, interracial relationship, sin...",[Forest Whitaker],"[Ronald Bass, Ezra Swerdlow, Deborah Schindler...","[Whitney Houston, Angela Bassett, Loretta Devi..."
4,False,Father of the Bride Collection,0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[baby, midlife crisis, confidence, aging, daug...","[Elliot Davis, Charles Shyer]",[Nancy Meyers],"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46120,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,[tragic love],"[Hamid Nematollah, Babak Ardalan]",[Hamid Nematollah],"[Leila Hatami, Kourosh Tahami, Elham Korda]"
46121,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,Released,NaN,Century of Birthing,False,9.0,3.0,"[artist, play, pinoy]",[Lav Diaz],[],"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
46122,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,Released,A deadly game of wits.,Betrayal,False,3.8,6.0,NaN,"[Mark L. Lester, João Fernandes]",[],"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
46123,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,Released,NaN,Satan Triumphant,False,0.0,0.0,NaN,[Yakov Protazanov],[Joseph N. Ermolieff],"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


In [7]:

### Note: I mistook this for ratings at first, so commenting out, the code might be useful later ###

#Clean up movies dataset:
#* remove movies with no rating information
#* remove movies with < 10 ratings
#print('Movies Shape before: ', movies.shape)
#movies = movies[(movies['vote_count'].notnull()) & (movies['vote_count'] > 9)]
#print('Movies Shape after: ', movies.shape)


In [8]:
ratings_small

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


# Data Understanding 2

Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.

# Modeling and Evaluation 1

Train and adjust parameters


# Modeling and Evaluation 2

Evaluate and Compare

# Modeling and Evaluation 3

Visualize Results

# Modeling and Evaluation 4

Summarize the Ramifications

# Deployment

Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?

# Exceptional Work

You have free reign to provide additional analyses or combine analyses.

# References

1. Dataset obtained from kaggle : <a href="https://www.kaggle.com/rounakbanik/the-movies-dataset"> click here </a>
2. Apple (turicreate) <a href="https://apple.github.io/turicreate/docs/userguide/recommender/"> article </a> on Recommender. 
3. Recommendor Systems instruction by Dr. Eric Larson: <a href="https://www.youtube.com/watch?v=I-xPNMCqW2o"> video </a>
4. Movie Recommendation System with Collaborative Filteringn by Code Heroku: <a href="https://www.youtube.com/watch?v=3ecNC-So0r4&t=903s"> video </a>